In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lbm_core import LBMSolver
from boundaries import TunnelBoundaries
from analysis import plot_complexity_dashboard
from aerodynamics import calculate_lift_drag, check_ground_effect


In [ ]:
# --- CONFIGURATION ---
NX, NY = 500, 200
REYNOLDS = 10000        # High Re to force turbulence
GROUND_TYPE = "no_slip" 

# --- SETUP ---
print(f"Initializing Virtual Wind Tunnel (Re={REYNOLDS}, Ground={GROUND_TYPE})...")
solver = LBMSolver(NX, NY, REYNOLDS, u_inlet=0.1)
bounds = TunnelBoundaries(NX, NY)

bounds.add_ground(type=GROUND_TYPE)
bounds.add_f1_wing_proxy(
    x_pos=50,
    height=80,
    length=40,
    slope=0.5
)


In [ ]:
# --- HISTORY LISTS (For the Plot) ---
drag_history = []
lift_history = []
step_history = []

# --- MAIN LOOP ---
print("Starting Simulation...")
TOTAL_STEPS = 3000

for step in range(TOTAL_STEPS + 1):
    
    # 1. PHYSICS
    solver.collide_and_stream(bounds.mask)
    bounds.apply_inlet_outlet(solver)
    
    # 2. AERO MONITOR & LOGGING
    if step % 100 == 0:
        fx, fy = calculate_lift_drag(solver, bounds)
        check_ground_effect(fx, fy)
        print(f"Step {step}/{TOTAL_STEPS}")
        
        # Save for plotting
        drag_history.append(fx)
        lift_history.append(fy)
        step_history.append(step)
        
    # 3. COMPLEXITY DASHBOARD
    if step > 0 and step % 500 == 0:
        print(f"   --> Displaying Complexity Dashboard (Step {step})...")
        plot_complexity_dashboard(solver, step)



In [ ]:
# --- FINAL VISUALIZATION 1: FLOW FIELD ---
print("Simulation Complete. Saving Visualizations...")

plt.figure(figsize=(12, 5), dpi=150)
velocity_mag = np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2)
velocity_mag[bounds.mask] = np.nan 
plt.imshow(velocity_mag, origin='lower', cmap='magma')
plt.colorbar(label="Flow Velocity |u|")
plt.title(f"Final Flow State (Re={REYNOLDS})")
plt.savefig("final_flow.png")
plt.show()


In [ ]:
# --- FINAL VISUALIZATION 2: AERO FORCES (NEW!) ---
plt.figure(figsize=(10, 5), dpi=100)
plt.plot(step_history, drag_history, 'r-', label='Drag (Fx)')
plt.plot(step_history, lift_history, 'b-', label='Lift (Fy)')
plt.axhline(0, color='black', linestyle='--', alpha=0.5) # Zero line
plt.xlabel("Simulation Step")
plt.ylabel("Force (LBM Units)")
plt.title(f"Aerodynamic Load History\n(Negative Lift = Downforce)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig("aero_forces_history.png") # Send this to your group!
print("Saved 'aero_forces_history.png'.")
plt.show()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.patches as patches
from lbm_core import LBMSolver
from boundaries import TunnelBoundaries
from aerodynamics import calculate_lift_drag

# --- CONFIGURATION ---
NX, NY = 800, 200
REYNOLDS = 10_000        # High speed turbulence
FRAMES = 6000            # Total video frames
STEPS_PER_FRAME = 45    # Speed up factor (sim steps per video frame)

# --- SETUP SIMULATION ---
print("Initializing Simulation for Video Rendering...")
solver = LBMSolver(NX, NY, REYNOLDS, u_inlet=0.1)
bounds = TunnelBoundaries(NX, NY)
bounds.add_ground(type="no_slip")
bounds.add_f1_wing_proxy(
    x_pos=150,
    height=19,
    length=60,
    slope=0.45
)
bounds.add_rectangular_obstacle(
    x_start=210,
    y_start=20,
    length=120,
    height=20
)
bounds.add_reverse_triangle(
    x_pos=240,
    height=39,
    length=90,
    slope=2/9
)
bounds.add_f1_wing_proxy(
    x_pos=300,
    height=49,
    length=30,
    slope=0.2
)

relevant_x_start = 145
relevant_x_end = 335
relevant_y_start = 5
relevant_y_end = 75

# --- SETUP PLOTS ---
fig = plt.figure(figsize=(16, 9), dpi=300)
gs = fig.add_gridspec(2, 1, height_ratios=[2, 1])

# Plot 1: The Flow (Top)
ax_flow = fig.add_subplot(gs[0])
# Initialize with zero velocity
velocity_mag = np.zeros((NY, NX))
velocity_mag[bounds.mask] = np.nan
im_flow = ax_flow.imshow(velocity_mag, origin='lower', cmap='magma', vmin=0, vmax=0.15)
ax_flow.set_title(f"F1 Ground Effect Turbulence (Re={REYNOLDS})")
# ax_flow.set_xticks([]) # Hide x-axis for cleaner look
ax_flow.add_patch(
    patches.Rectangle(
        (relevant_x_start, relevant_y_start), relevant_x_end - relevant_x_start, relevant_y_end - relevant_y_start,
        linewidth=1, edgecolor='cyan', facecolor='none', linestyle='--'
    )
)
plt.colorbar(im_flow, ax=ax_flow, label="Speed |u|", orientation="horizontal", pad=0.2)

# Plot 2: The Force Monitor (Bottom)
ax_force = fig.add_subplot(gs[1])
line_lift, = ax_force.plot([], [], 'b-', linewidth=2, label='Lift (Downforce)')
line_drag, = ax_force.plot([], [], 'r-', linewidth=2, label='Drag')
ax_force.set_xlim(0, FRAMES)
ax_force.set_ylim(-3.0, 1.0) # Adjust based on your previous logs
ax_force.set_xlabel("Video Frame")
ax_force.set_ylabel("Force")
ax_force.legend(loc="upper right")
ax_force.grid(True, alpha=0.3)
ax_force.set_title("Real-Time Aerodynamic Load")

# Data containers
lift_data = []
drag_data = []
frame_indices = []

def update(frame):
    # Run physics steps
    for _ in range(STEPS_PER_FRAME):
        solver.collide_and_stream(bounds.mask)
        bounds.apply_inlet_outlet(solver)
    
    # Calculate Aero
    fx, fy = calculate_lift_drag(
        solver,
        bounds,
        x_start=relevant_x_start,
        x_end=relevant_x_end,
        y_start=relevant_y_start,
        y_end=relevant_y_end
    )
    
    # Update Data
    lift_data.append(fy)
    drag_data.append(fx)
    frame_indices.append(frame)
    
    # Update Flow Image
    v_mag = np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2)
    v_mag[bounds.mask] = np.nan
    im_flow.set_array(v_mag)
    
    # Update Force Lines
    line_lift.set_data(frame_indices, lift_data)
    line_drag.set_data(frame_indices, drag_data)
    
    if frame % 10 == 0:
        print(f"Rendering Frame {frame}/{FRAMES}")
    
    return im_flow, line_lift, line_drag


print("Starting Render... (This might take a minute)")

# split into partial GIFs to manage memory
for i in range(0, FRAMES, 150):
    end_frame = min(i + 150, FRAMES)
    print(f"Rendering frames {i} to {end_frame}...")
    partial_anim = FuncAnimation(fig, update, frames=range(i, end_frame), interval=50, blit=False)
    partial_anim.save(f'partial_anim/partial_f1_turbulence_{i}_{end_frame}.gif', writer='pillow', fps=30)
    plt.close(fig)
print("Rendering complete. Saving final animation...")

Initializing Simulation for Video Rendering...
Starting Render... (This might take a minute)
Rendering frames 0 to 150...
Rendering Frame 0/6000
Rendering Frame 0/6000
Rendering Frame 10/6000
Rendering Frame 20/6000
Rendering Frame 30/6000
Rendering Frame 40/6000
Rendering Frame 50/6000
Rendering Frame 60/6000
Rendering Frame 70/6000
Rendering Frame 80/6000
Rendering Frame 90/6000
Rendering Frame 100/6000
Rendering Frame 110/6000
Rendering Frame 120/6000
Rendering Frame 130/6000
Rendering Frame 140/6000
Rendering frames 150 to 300...
Rendering Frame 150/6000
Rendering Frame 150/6000
Rendering Frame 160/6000
Rendering Frame 170/6000
Rendering Frame 180/6000
Rendering Frame 190/6000
Rendering Frame 200/6000
Rendering Frame 210/6000
Rendering Frame 220/6000
Rendering Frame 230/6000
Rendering Frame 240/6000
Rendering Frame 250/6000
Rendering Frame 260/6000
Rendering Frame 270/6000
Rendering Frame 280/6000
Rendering Frame 290/6000
Rendering frames 300 to 450...
Rendering Frame 300/6000
Rend